# Adding Visual Features

### Step 1: Coordinate Extraction

In [121]:
# Ines's code takes PDF and yields pdf_word_list, coordinate_map
import os 
import subprocess

pdf_file = "/Users/ineschami/Desktop/bc546-d.pdf" # Path to PDF file 
# create coordinates and id files
os.system('./getCoordinates.sh {}'.format(pdf_file)) # requires bs4 and lxml pareser installed

basename = subprocess.check_output('basename {} .pdf'.format(pdf_file), shell=True)
dirname = subprocess.check_output('dirname {}'.format(pdf_file), shell=True)
coordinates = open(dirname.rstrip() + '/' + basename.rstrip() + '.ids_coordinates.txt','r').readlines()
words = open(dirname.rstrip() + '/' + basename.rstrip() + '.ids_words.txt','r').readlines()
words_map = {}
coordinate_map= {}
for i in range(len(words)):
    try:
        # get words and ids
        word_id, word = words[i].rstrip().split('\t')
        words_map[word_id] = word
        # get ids and coordinates 
        word_id, page_nb, top, left, bottom, right = coordinates[i].rstrip().split('\t')
        coordinate_map[word_id] = (page_nb, top, left, bottom, right)
    except ValueError as e: 
        if str(e) == 'need more than 1 value to unpack': # word is a white space, ignore it
            pass
        else: 
            raise

### Step 2: PDF to HTML Conversion

Use Adobe Acrobat (or other program of your choice) to convert PDF -> HTML with structure.

### Step 3: HTML Parsing

In [ ]:
# Payal's parser takes HTML and yields corpus object, html_word_list

### Step 4: Visual Linking

In [ ]:
from visual_linking import link_lists

%time links = link_lists(pdf_word_list, html_word_list)

### Step 5: Updating with coordinates

In [2]:
# Payal's code walks through phrases, updating each one's five visual attributes
# (page, top, left, bottom, right)

### Step 6: Assess Results

In [122]:
# Use Ines's code to plot locations of words on the original PDF
import numpy as np
import cv2

In [128]:
page_to_visualize = '2' # page number to visualize

In [129]:
# Create a black image
img = np.zeros((1150,920,3), np.uint8)  # size of A4 pdf 
for page_nb, top, left, bottom, right in coordinate_map.values():
    if page_nb == page_to_visualize:
        cv2.rectangle(img,(int(left),int(top)),(int(right),int(bottom)),(0,255,0),3)

In [130]:
cv2.imshow('Bounding boxes',img)
cv2.waitKey() # press any key to exit the opencv output 
cv2.destroyAllWindows() 

In [131]:
cv2.imwrite('/Users/ineschami/Desktop/test.png', img)

True